In [71]:
import os
from model_baselines import BaselineClassifier

from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from typing import List, Dict, Optional, Tuple, Any
import numpy as np
from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, BatchNormalization
import tensorflow as tf
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import KFold
from sklearn.model_selection import StratifiedKFold

In [87]:
final_arr1 = np.load(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_and_chich_split_text_embeddings' '.npy')
final_arr2 = np.load(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_and_chich_split_translated_text' '.npy')
print(final_arr1.shape)
print(final_arr2.shape)
final_arr=np.hstack([final_arr1,final_arr2])
final_arr=final_arr1

labels_df=pd.read_csv(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_and_chich_split_labels'+'.csv')
target_names=labels_df['Label'].unique().astype(str).tolist()

labels=labels_df['Label'].values

idx=pd.read_csv(f'/projectnb2/cs505/students/lseoane/malawi_news_classification/data/'+'eng_and_chich_split_index'+'.csv')
idx=idx['Idx'].values


(8541, 512)
(8541, 512)


In [88]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()
le.fit(target_names)

f_target_names=le.transform(target_names)
labels=le.transform(labels)

In [89]:
def get_model(size):
    print('Building model...')
    model = Sequential()
    model.add(Dense(2048, input_shape=(size,)))
    model.add(Activation('relu'))
    model.add(Dropout(0.2))
    model.add(Dense(20))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [90]:
def get_results(model,epochs,train_dataset,test_dataset,y_test):
    model.fit(train_dataset,epochs=epochs, batch_size=32,verbose=1)
    print(model.evaluate(test_dataset))
    ans = model.predict(test_dataset)
    print(accuracy_score(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))
    print(classification_report(y_test, np_utils.to_categorical(np.argmax(ans,axis=1),num_classes=20)))



In [79]:
# y=np_utils.to_categorical(labels, 20)

In [92]:
X=final_arr
y=labels

skf = StratifiedKFold(n_splits=3,random_state=np.random.seed(42))
skf.get_n_splits(X,y)

for train_index, test_index in skf.split(X, y):
    X_train, X_test = X[train_index], X[test_index]
    y_train, y_test = y[train_index], y[test_index]
    y_train=np_utils.to_categorical(y_train, 20)
    y_test=np_utils.to_categorical(y_test, 20)
    
    train_dataset = tf.data.Dataset.from_tensor_slices((X_train, y_train))
    test_dataset = tf.data.Dataset.from_tensor_slices((X_test, y_test))
    
    BATCH_SIZE = 32
    SHUFFLE_BUFFER_SIZE = 100

    train_dataset = train_dataset.shuffle(SHUFFLE_BUFFER_SIZE).batch(BATCH_SIZE)
    test_dataset = test_dataset.batch(BATCH_SIZE)
    get_results(get_model(512,),50,train_dataset,test_dataset,y_test)

Building model...
Epoch 1/50
178/178 [==============================] - 1s 5ms/step - loss: 2.5910 - accuracy: 0.2292
Epoch 2/50
178/178 [==============================] - 1s 5ms/step - loss: 2.3229 - accuracy: 0.3021
Epoch 3/50
178/178 [==============================] - 1s 5ms/step - loss: 2.1705 - accuracy: 0.3397
Epoch 4/50
178/178 [==============================] - 1s 5ms/step - loss: 2.0577 - accuracy: 0.3653
Epoch 5/50
178/178 [==============================] - 1s 5ms/step - loss: 2.0009 - accuracy: 0.3751
Epoch 6/50
178/178 [==============================] - 1s 5ms/step - loss: 1.9551 - accuracy: 0.3918
Epoch 7/50
178/178 [==============================] - 1s 5ms/step - loss: 1.9077 - accuracy: 0.4029
Epoch 8/50
178/178 [==============================] - 1s 5ms/step - loss: 1.8722 - accuracy: 0.4153
Epoch 9/50
178/178 [==============================] - 1s 5ms/step - loss: 1.8340 - accuracy: 0.4271
Epoch 10/50
178/178 [==============================] - 1s 5ms/step - loss: 1.8050 

178/178 [==============================] - 1s 5ms/step - loss: 1.6299 - accuracy: 0.4879
Epoch 18/50
178/178 [==============================] - 1s 5ms/step - loss: 1.6252 - accuracy: 0.4842
Epoch 19/50
178/178 [==============================] - 1s 5ms/step - loss: 1.5973 - accuracy: 0.4909
Epoch 20/50
178/178 [==============================] - 1s 5ms/step - loss: 1.5688 - accuracy: 0.5035
Epoch 21/50
178/178 [==============================] - 1s 5ms/step - loss: 1.5475 - accuracy: 0.5067
Epoch 22/50
178/178 [==============================] - 1s 5ms/step - loss: 1.5245 - accuracy: 0.5221
Epoch 23/50
178/178 [==============================] - 1s 5ms/step - loss: 1.4986 - accuracy: 0.5204
Epoch 24/50
178/178 [==============================] - 1s 5ms/step - loss: 1.4822 - accuracy: 0.5311
Epoch 25/50
178/178 [==============================] - 1s 5ms/step - loss: 1.4528 - accuracy: 0.5395
Epoch 26/50
178/178 [==============================] - 1s 5ms/step - loss: 1.4243 - accuracy: 0.5429
Ep

178/178 [==============================] - 1s 5ms/step - loss: 1.2358 - accuracy: 0.6038
Epoch 34/50
178/178 [==============================] - 1s 5ms/step - loss: 1.2108 - accuracy: 0.6098
Epoch 35/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1921 - accuracy: 0.6161
Epoch 36/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1661 - accuracy: 0.6275
Epoch 37/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1545 - accuracy: 0.6319
Epoch 38/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1293 - accuracy: 0.6289
Epoch 39/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1173 - accuracy: 0.6403
Epoch 40/50
178/178 [==============================] - 1s 5ms/step - loss: 1.1030 - accuracy: 0.6396
Epoch 41/50
178/178 [==============================] - 1s 5ms/step - loss: 1.0653 - accuracy: 0.6561
Epoch 42/50
178/178 [==============================] - 1s 5ms/step - loss: 1.0571 - accuracy: 0.6563
Ep

/share/pkg.7/python3/3.8.6/install/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
